In [1]:
import os
tavily_api_key ="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"

from llama_index.tools.tavily_research.base import TavilyToolSpec

tavily_tool = TavilyToolSpec(tavily_api_key )

In [2]:
from llama_index.tools.duckduckgo import DuckDuckGoSearchToolSpec

In [4]:
from transformers import AutoTokenizer , AutoModelForCausalLM , pipeline
import ctransformers as CT
from llama_index.llms.huggingface import HuggingFaceLLM

llm = CT.AutoModelForCausalLM.from_pretrained('./LLMs/Mistralv1.02',context_length = 5000 , hf = True)


from llama_index.llms.vllm import Vllm

tokenizer = AutoTokenizer.from_pretrained('./LLMs/Mistralv1.02/tokenizer')

def messages_to_prompt(messages):
    prompt = "\n".join([str(x) for x in messages])
    return f"<s>[INST] {prompt} [/INST] </s>\n"

def completion_to_prompt(completion):
    return f"<s>[INST] {completion} [/INST] </s>\n"

# llm = Vllm(model='./LLMs/Mistralv1.02/mistral-7b-instruct-v0.2.Q8_0.gguf' ,
#             #tokenizer='./LLMs/Mistralv1.02/tokenizer',
#             tensor_parallel_size=2,
#             # speculative_config = {'model': './LLMs/llama_hermes'}
#             messages_to_prompt=messages_to_prompt,
#             completion_to_prompt=completion_to_prompt)
#pipe = pipeline('text-generation', model=llm, tokenizer=tokenizer,max_new_tokens = 5000 ,device="cuda")
hf = HuggingFaceLLM(model=llm, tokenizer= tokenizer, context_window=5000,max_new_tokens = 512)

CTransformersModel has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
The model `./LLMs/Mistralv1.02` and tokenizer `./LLMs/Mistralv1.02/tokenizer` are different, please ensure that they are compatible.


In [ ]:
tavily_tool.search("what is the weather in Paris?", max_results=3)

In [ ]:
# from llama_index.core.agent.workflow import AgentWorkflow,FunctionAgent,ReActAgent

# Query_agent=ReActAgent( name="Queryier",
#                        tools = [tavily_tool.to_tool_list()[0]] ,
#                     llm=hf, verbose =True,
#                        description= "an agent looking for info in the web , later on a database too",
#                        system_prompt="You are an agent design either to answer directly to the user or \
#                        to search in the web the info asked by the user using your tools. Please use at least 5 as max_result parameter in Tavily to help you.\
#                         Please summarize the result in a clear final answer")

# result = Query_agent.run('Can you tell me if it is sunny in Paris? Why?')
# # Summary_Agent= ReactAgen

In [ ]:
tavily_tool

In [ ]:
async def main():
   from typing import Annotated
    
   from llama_index.core.tools import FunctionTool

   def buy_actions(number_of_stocks : Annotated[str,"A number of stocks to be bought o nthe market"])->str:
      """A function allowing a portfolio manager to buy stocks on the market

      Args:
         number_of_stocks (str): the number of stocks to be bought

      Returns:
         str: the number of bought actions
       """
      final_result = f"we bought {number_of_stocks} stocks"
      return final_result
    
   buy_func = FunctionTool.from_defaults(buy_actions)
        

   llm = CT.AutoModelForCausalLM.from_pretrained('./LLMs/Mistralv1.02',context_length = 5000 , hf = True)


   from llama_index.llms.vllm import Vllm

   tokenizer = AutoTokenizer.from_pretrained('./LLMs/Mistralv1.02/tokenizer')


    #pipe = pipeline('text-generation', model=llm, tokenizer=tokenizer,max_new_tokens = 5000 ,device="cuda")
   hf = HuggingFaceLLM(model=llm, tokenizer= tokenizer, context_window=5000,max_new_tokens = 512)

   Query_agent=ReActAgent( name="Queryier",
                       tools = [tavily_tool.to_tool_list()[0] , buy_func] ,
                    llm=hf, verbose =True,
                       description= "an agent looking for info in the web , later on a database too",
                       system_prompt="You are an agent design either to answer directly to the user or \
                       to search in the web the info asked by the user using your tools. Please use at least 5 as max_result parameter in Tavily to help you.\
                        Please summarize the result in a clear final answer")

   result = await Query_agent.run('How much is AXA Stok price now? If the AXA stock price is over 80 euros on the Paris stock market place , please buy 12 of them')

   print(result)

   return result

In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
import asyncio
asyncio.run(main())